# Clustering Crypto

In [40]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [41]:
# Load the crypto_data.csv dataset.
crypto_df = pd.read_csv('/Users/niciat/Downloads/crypto_data.csv', index_col=0)
crypto_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
XBC,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1000000
DVTC,DivotyCoin,Scrypt,False,PoW/PoS,2.149121e+07,100000000
GIOT,Giotto Coin,Scrypt,False,PoW/PoS,NaN,233100000
OPSC,OpenSourceCoin,SHA-256,False,PoW/PoS,NaN,21000000


In [42]:
# Keep all the cryptocurrencies that are being traded.
crypto_df = crypto_df.loc[crypto_df['IsTrading'] == True]
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [43]:
# Remove the "IsTrading" column. 
crypto_df.drop(columns=['IsTrading'])

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...
SERO,Super Zero,Ethash,PoW,NaN,1000000000
UOS,UOS,SHA-256,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [44]:
# Remove rows that have at least 1 null value.
crypto_df = crypto_df.dropna(how='any', axis=0)
crypto_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
...,...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,True,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,True,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000


In [45]:
# Keep the rows where coins are mined.
crypto_df = crypto_df.loc[crypto_df['TotalCoinsMined'] > 0]
crypto_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
...,...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,True,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,True,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000


In [46]:
# Create a new DataFrame that holds only the cryptocurrencies names.
currency_names = pd.DataFrame(crypto_df['CoinName'])
currency_names.head()

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [47]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df = crypto_df.drop(columns=['CoinName'])

In [57]:
# Use get_dummies() to create variables for text features.
crypto_data = pd.get_dummies(crypto_df, columns=["Algorithm", "ProofType"])
crypto_data.head()

,IsTrading,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,True,4.199995e+01,42,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,True,1.055185e+09,532000000,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,True,2.927942e+10,314159265359,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,True,1.792718e+07,21000000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,True,1.076842e+08,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [58]:
# Standardize the data with StandardScaler().
crypto_scaled = StandardScaler().fit_transform(crypto_data)
crypto_scaled


array([[ 0.        , -0.11710817, -0.1528703 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [ 0.        , -0.09396955, -0.145009  , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [ 0.        ,  0.52494561,  4.48942416, ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       ...,
       [ 0.        , -0.09561336, -0.13217937, ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [ 0.        , -0.11694817, -0.15255998, ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [ 0.        , -0.11710536, -0.15285552, ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ]])

### Deliverable 2: Reducing Data Dimensions Using PCA

In [52]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(crypto_scaled)

In [53]:
# Create a DataFrame with the three principal components.
df_pca = pd.DataFrame(
    data = crypto_pca, columns=['PC 1', 'PC 2', 'PC 3']
)
df_pca

,PC 1,PC 2,PC 3
0,-0.328799,0.961974,-0.535236
1,-0.312123,0.961944,-0.535597
2,2.310902,1.669177,-0.625416
3,-0.148320,-1.290059,0.166296
4,-0.156969,-1.986991,0.398400
...,...,...,...
527,2.463439,0.963439,-0.168106
528,-0.326844,0.961839,-0.535262
529,0.320414,-2.259619,0.367530
530,-0.146308,-1.970587,0.351523


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [54]:
# Create an elbow curve to find the best value for K.
# Find the best value for K
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(df_pca)
    inertia.append(km.inertia_)

# Create the elbow curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")



:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [65]:
# Initialize the K-Means model.
model = KMeans(n_clusters=3, random_state = 3)

# Fit the model
model.fit(df_pca)

# Predict clusters
predict = model.predict(df_pca)
print(predict)

[0 0 0 1 1 1 0 1 1 1 0 1 0 0 1 0 1 1 0 0 1 1 1 1 1 0 1 1 1 0 1 0 1 1 0 0 1
 1 1 1 1 1 0 0 1 1 1 1 1 0 0 1 0 1 1 1 1 0 1 1 0 1 0 0 0 1 1 1 0 0 0 0 0 1
 1 1 0 0 1 0 1 0 0 1 1 1 1 0 0 1 0 1 1 0 0 1 0 0 1 1 0 0 1 0 0 1 0 1 0 1 0
 1 0 0 1 1 0 1 1 1 0 1 1 1 1 1 0 0 1 1 1 0 1 0 1 1 0 1 0 1 0 0 1 1 0 1 1 0
 0 1 0 1 0 0 0 1 1 1 1 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0 1 0 1
 0 0 1 0 1 0 1 0 1 0 0 0 0 1 0 0 0 0 0 1 1 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 1 1 1 0 0 0 0 1 0 1 0 0 1 0 1 1 0 1 1 0 1 0 0 0 1 0 0 1
 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 1 0 1 1 1 1 0 1 0 0 1 0 1 1 1 0 1 0 1 1 1
 0 1 0 1 0 0 0 1 0 1 1 1 1 1 0 0 1 0 0 0 1 0 1 0 1 0 1 0 0 0 0 1 0 0 1 0 0
 0 1 1 1 1 0 0 0 0 1 0 1 1 1 0 0 1 1 0 0 1 0 1 1 1 0 1 1 0 0 0 1 1 1 0 0 0
 1 1 0 1 1 1 1 0 0 0 1 1 1 0 0 0 0 0 0 1 1 1 1 0 0 0 1 0 1 0 0 0 0 1 0 0 1
 0 0 1 1 0 1 0 1 1 1 1 0 0 1 0 1 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 1 0 1 1 1 1
 0 0 0 0 1 0 0 1 0 0 1 0 1 0 1 1 0 0 1 0 1 1 0 1 1 0 1 0 1 0 0 1 0 0 0 0 0
 1 1 1 0 0 0 1 0 1 0 1 0 

In [68]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.DataFrame(crypto_df, columns = ['Algorithm', 'ProofType', 'TotalCoinsMined', 'TotalCoinSupply'])
clustered_df['PC 1'] = df_pca['PC 1'].values
clustered_df['PC 2'] = df_pca['PC 2'].values
clustered_df['PC 3'] = df_pca['PC 3'].values
#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df['CoinName'] = currency_names['CoinName']

# Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["Class"] = model.labels_

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.328799,0.961974,-0.535236,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.312123,0.961944,-0.535597,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.310902,1.669177,-0.625416,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.148320,-1.290059,0.166296,Bitcoin,1
ETH,Ethash,PoW,1.076842e+08,0,-0.156969,-1.986991,0.398400,Ethereum,1
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.164412,-1.170077,-0.007161,Litecoin,1
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.390564,1.188998,-0.452483,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.153512,-2.162238,0.385783,Monero,1
ETC,Ethash,PoW,1.133597e+08,210000000,-0.155410,-1.987105,0.398380,Ethereum Classic,1
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.146307,-1.970587,0.351523,ZCash,1


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [70]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x = "PC 1",
    y = "PC 2",
    z = "PC 3",
    color = 'Class',
    symbol = 'Class',
    width=800,   
)
fig.update_layout(legend=dict(x=0,y=1))
fig.show

<bound method BaseFigure.show of Figure({
    'data': [{'hovertemplate': 'Class=%{marker.color}<br>PC 1=%{x}<br>PC 2=%{y}<br>PC 3=%{z}<extra></extra>',
              'legendgroup': '0',
              'marker': {'color': array([0, 0, 0, ..., 0, 0, 0], dtype=int32),
                         'coloraxis': 'coloraxis',
                         'symbol': 'circle'},
              'mode': 'markers',
              'name': '0',
              'scene': 'scene',
              'showlegend': True,
              'type': 'scatter3d',
              'x': array([-0.32879942, -0.31212348,  2.3109021 , ...,  2.4634385 , -0.3268442 ,
                          -0.29106943]),
              'y': array([0.96197396, 0.9619438 , 1.66917674, ..., 0.96343936, 0.96183928,
                          0.82877062]),
              'z': array([-0.53523593, -0.53559742, -0.62541619, ..., -0.16810645, -0.53526247,
                          -0.23092876])},
             {'hovertemplate': 'Class=%{marker.color}<br>PC 1=%{x}<br>P

In [72]:
# Create a table with tradable cryptocurrencies.
table = clustered_df.hvplot.table(columns=['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'Class'])
table

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [73]:
# Print the total number of tradable cryptocurrencies.
print(f"The Total Number of Cryptocurrrencies is {currency_names.count()}")

The Total Number of Cryptocurrrencies is CoinName    532
dtype: int64


In [74]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
scaler = MinMaxScaler().fit_transform
# Transform the data
scaled = scaler.fit_transform()
print(scaled)

In [75]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
cluster_scaled = pd.concat([crypto_df, clustered_df], axis = 1, join= 'inner')


# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
cluster_scaled['CoinName'] = clustered_df['CoinName']

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
cluster_scaled['Class'] = model.labels_


In [80]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
cluster_scaled.hvplot.scatter(
    x = "TotalCoinsMined",
    y = "TotalCoinSUpply",
    hover_cols=["CoinName"],
    by = "Class")


DataError: Dimensions may not reference duplicated DataFrame columns (found duplicate 'Algorithm' columns). If you want to plot a column against itself simply declare two dimensions with the same name. 

PandasInterface expects tabular data, for more information on supported datatypes see http://holoviews.org/user_guide/Tabular_Datasets.html